In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

In [2]:
import os # 모델 저장할 폴더가 있는지 여부와 폴더 생성
import numpy as np
import pandas as pd # get_dummies, crosstab
import seaborn as sns # iris 데이터 가져오기
from sklearn import datasets # iris 데이터 (X,y) 가져오기
from sklearn.preprocessing import LabelEncoder # 라벨인코더
from tensorflow.keras.utils import to_categorical # 원핫인코딩
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Input, Dropout, LeakyReLU
from tensorflow.keras import metrics # compile
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
import matplotlib.pyplot as plt

# 1. 기본적인 DNN

In [3]:
# 데이터 생성 및 전처리
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y,
                                                    test_size=0.2,
                                                    stratify= iris_y)
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [4]:
# 2. 모델구성
model = Sequential([
    Input(4),
    Dense(units=64, activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=50, activation='relu'),
    Dense(units=30, activation='relu'),
    Dense(units=3, activation='softmax')    
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 50)                6450      
                                                                 
 dense_3 (Dense)             (None, 30)                1530      
                                                                 
 dense_4 (Dense)             (None, 3)                 93        
                                                                 
Total params: 16,713
Trainable params: 16,713
Non-trainable params: 0
_________________________________________________________________


In [5]:
# 3. 학습과정 설정
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# 4. 모델 학습
earlystopping = EarlyStopping(patience=40)# monitor='val_loss' 기본값
hist=model.fit(train_X,train_y,
               epochs=200,
               validation_split=0.1,
               callbacks=[earlystopping])

Epoch 1/200
4/4 [==============================] - 1s 70ms/step - loss: 1.0675 - accuracy: 0.3333 - val_loss: 0.9650 - val_accuracy: 0.3333
Epoch 2/200
4/4 [==============================] - 0s 12ms/step - loss: 0.9513 - accuracy: 0.7593 - val_loss: 0.8765 - val_accuracy: 1.0000
Epoch 3/200
4/4 [==============================] - 0s 13ms/step - loss: 0.8575 - accuracy: 0.8241 - val_loss: 0.7861 - val_accuracy: 0.9167
Epoch 4/200
4/4 [==============================] - 0s 11ms/step - loss: 0.7573 - accuracy: 0.9167 - val_loss: 0.6804 - val_accuracy: 1.0000
Epoch 5/200
4/4 [==============================] - 0s 13ms/step - loss: 0.6710 - accuracy: 0.9815 - val_loss: 0.6005 - val_accuracy: 1.0000
Epoch 6/200
4/4 [==============================] - 0s 11ms/step - loss: 0.5850 - accuracy: 0.9537 - val_loss: 0.5355 - val_accuracy: 1.0000
Epoch 7/200
4/4 [==============================] - 0s 49ms/step - loss: 0.5096 - accuracy: 0.9537 - val_loss: 0.4495 - val_accuracy: 1.0000
Epoch 8/200
4/4 [===

4/4 [==============================] - 0s 10ms/step - loss: 0.0713 - accuracy: 0.9630 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 60/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0458 - accuracy: 0.9815 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 61/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0365 - accuracy: 0.9907 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 62/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0367 - accuracy: 0.9907 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 63/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0429 - accuracy: 0.9907 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 64/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0325 - accuracy: 0.9907 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 65/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0593 - accuracy: 0.9722 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 66/200
4/4 [========

4/4 [==============================] - 0s 10ms/step - loss: 0.0528 - accuracy: 0.9722 - val_loss: 4.8548e-04 - val_accuracy: 1.0000
Epoch 117/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0385 - accuracy: 0.9722 - val_loss: 4.7696e-04 - val_accuracy: 1.0000
Epoch 118/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0279 - accuracy: 0.9815 - val_loss: 4.4584e-04 - val_accuracy: 1.0000
Epoch 119/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0288 - accuracy: 1.0000 - val_loss: 5.2831e-04 - val_accuracy: 1.0000
Epoch 120/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0291 - accuracy: 0.9815 - val_loss: 4.4997e-04 - val_accuracy: 1.0000
Epoch 121/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0334 - accuracy: 0.9815 - val_loss: 3.8314e-04 - val_accuracy: 1.0000
Epoch 122/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0275 - accuracy: 0.9907 - val_loss: 3.7427e-04 - val_accuracy:

4/4 [==============================] - 0s 10ms/step - loss: 0.0188 - accuracy: 1.0000 - val_loss: 1.4138e-04 - val_accuracy: 1.0000
Epoch 173/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0404 - accuracy: 0.9722 - val_loss: 1.3028e-04 - val_accuracy: 1.0000
Epoch 174/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0267 - accuracy: 0.9815 - val_loss: 1.3185e-04 - val_accuracy: 1.0000
Epoch 175/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0172 - accuracy: 0.9907 - val_loss: 1.6352e-04 - val_accuracy: 1.0000
Epoch 176/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0324 - accuracy: 0.9815 - val_loss: 1.3599e-04 - val_accuracy: 1.0000
Epoch 177/200
4/4 [==============================] - 0s 14ms/step - loss: 0.0169 - accuracy: 1.0000 - val_loss: 1.3627e-04 - val_accuracy: 1.0000
Epoch 178/200
4/4 [==============================] - 0s 11ms/step - loss: 0.0340 - accuracy: 0.9815 - val_loss: 1.3282e-04 - val_accuracy:

# 2. sklearn 이용
- 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동

In [6]:
from sklearn.neural_network import MLPClassifier

In [7]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [8]:
model = MLPClassifier(
            hidden_layer_sizes=(64, 128, 50), # hidden layer의 units 수
            activation='relu',
            solver='adam', # sgd등의 optimizer
            batch_size=40,
            max_iter=1000, # 학습최대 횟수
            early_stopping=True, # 조기종료 활성화
            n_iter_no_change=10, # earlystopping의 patience와 유사
            validation_fraction=0.1, # 검증셋 비율
            warm_start=False # True일 경우 이전학습에 이어서 학습
)

In [9]:
model.fit(train_X,train_y)

MLPClassifier(batch_size=40, early_stopping=True,
              hidden_layer_sizes=(64, 128, 50), max_iter=1000)

In [10]:
model.score(test_X, test_y)

0.9

In [11]:
iris_X[0]

array([5.1, 3.5, 1.4, 0.2])

In [12]:
# 모델 사용
input_data = [[5.1, 3.5, 1.4, 0.2]]
model.predict(input_data)

array([0])

In [13]:
# 교차표
# test_y : 실제값
hat_y = model.predict(test_X)
pd.crosstab(test_y, hat_y, rownames=['real'], colnames=['predict'])

predict,0,1,2
real,,,
0,10,0,0
1,0,7,3
2,0,0,10


# 3. 클래스를 생성하여 모델함수 사용

In [22]:
class DNNClassifier:
    @staticmethod
    def build(input_dim=4, activation='relu', output_dim=3):
#         model = Sequential()
#         model.add(Input(input_dim))
#         model.add(Dense(units=50, activation=activation))
#         model.add(Dense(units=30, activation=activation))
#         model.add(Dense(units=3, activation='softmax'))
        # 모델구성
        model = Sequential([
            Input(input_dim),
            Dense(units=50, activation=activation),
            Dense(units=30, activation=activation),
            Dense(units=output_dim, activation='softmax'),
        ])
        # 학습설정
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        return model

In [15]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape
# 2. 모델
model = DNNClassifier.build(input_dim=4, activation='elu')
# 3. 모델학습
hist = model.fit(train_X, train_y, epochs=50, validation_split=0.1)

Epoch 1/50
4/4 [==============================] - 1s 53ms/step - loss: 1.6668 - accuracy: 0.3148 - val_loss: 0.9174 - val_accuracy: 0.5000
Epoch 2/50
4/4 [==============================] - 0s 13ms/step - loss: 1.0516 - accuracy: 0.3148 - val_loss: 0.9085 - val_accuracy: 0.5833
Epoch 3/50
4/4 [==============================] - 0s 12ms/step - loss: 0.9185 - accuracy: 0.6944 - val_loss: 1.0734 - val_accuracy: 0.5000
Epoch 4/50
4/4 [==============================] - 0s 12ms/step - loss: 0.9376 - accuracy: 0.6667 - val_loss: 1.0590 - val_accuracy: 0.5000
Epoch 5/50
4/4 [==============================] - 0s 13ms/step - loss: 0.8882 - accuracy: 0.6852 - val_loss: 0.9066 - val_accuracy: 0.5000
Epoch 6/50
4/4 [==============================] - 0s 14ms/step - loss: 0.8004 - accuracy: 0.6852 - val_loss: 0.7428 - val_accuracy: 0.5000
Epoch 7/50
4/4 [==============================] - 0s 11ms/step - loss: 0.7362 - accuracy: 0.8241 - val_loss: 0.6446 - val_accuracy: 0.9167
Epoch 8/50
4/4 [===========

In [16]:
# 모델 평가
loss, accuracy = model.evaluate(test_X, test_y)
print(f'정확도 : {accuracy*100}%')

1/1 [==============================] - 0s 23ms/step - loss: 0.2162 - accuracy: 0.9333
정확도 : 93.33333373069763%


# 4. 함수형 API
- 병렬처리, Residual block

In [28]:
# 기존 model 스타일1
# model = Sequential()
# model.add(Input(shape=(4,))),
# model.add(Dense(units=50, activation=activation))
# model.add(Dense(units=30, activation=activation))
# model.add(Dense(units=3, activation='softmax'))
#  기존 model 스타일2
model = Sequential([
    Input(shape=(4,)),
    Dense(units=50, activation='relu'),
    Dense(units=30, activation='relu'),
    Dense(units=3, activation='softmax')
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 50)                250       
                                                                 
 dense_22 (Dense)            (None, 30)                1530      
                                                                 
 dense_23 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [29]:
# 기존 model 스타일3 (함수형 API)
from tensorflow.keras import Model
input_=Input(shape=(4,))
layer1=Dense(units=50, activation='relu')(input_)
layer2=Dense(units=30, activation='relu')(layer1)
output=Dense(units=3, activation='softmax')(layer2)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_24 (Dense)            (None, 50)                250       
                                                                 
 dense_25 (Dense)            (None, 30)                1530      
                                                                 
 dense_26 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [30]:
# 병렬처리방식
from tensorflow.keras.layers import concatenate
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=80, activation='relu')(input_)
dense3 = Dense(units=30, activation='relu')(input_)
x = concatenate([dense1,dense2,dense3])
output = Dense(units=3, activation='softmax')(x)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_27 (Dense)               (None, 50)           250         ['input_10[0][0]']               
                                                                                                  
 dense_28 (Dense)               (None, 80)           400         ['input_10[0][0]']               
                                                                                                  
 dense_29 (Dense)               (None, 30)           150         ['input_10[0][0]']               
                                                                                            

In [31]:
# 리지듀얼 블럭(Residual Block) : 딥러닝에서 기울기 소실 문제로 학습이 잘 되지 않는 부분을 해결하기 위한 제안
from tensorflow.keras.layers import add
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=50, activation='relu')(dense1)
dense3 = add([dense1,dense2])
output = Dense(units=3, activation='softmax')(dense3)
model = Model(inputs=input_,outputs=output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_31 (Dense)               (None, 50)           250         ['input_11[0][0]']               
                                                                                                  
 dense_32 (Dense)               (None, 50)           2550        ['dense_31[0][0]']               
                                                                                                  
 add_2 (Add)                    (None, 50)           0           ['dense_31[0][0]',               
                                                                  'dense_32[0][0]']         

In [32]:
# 학습과정 설정 & 학습
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(train_X, train_y, epochs=200, verbose=1)

Epoch 1/200
4/4 [==============================] - 0s 2ms/step - loss: 1.6879 - accuracy: 0.3333
Epoch 2/200
4/4 [==============================] - 0s 3ms/step - loss: 1.3513 - accuracy: 0.1083
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 1.1735 - accuracy: 0.3333
Epoch 4/200
4/4 [==============================] - 0s 2ms/step - loss: 1.0785 - accuracy: 0.3417
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 1.0283 - accuracy: 0.4667
Epoch 6/200
4/4 [==============================] - 0s 2ms/step - loss: 0.9626 - accuracy: 0.5583
Epoch 7/200
4/4 [==============================] - 0s 3ms/step - loss: 0.8956 - accuracy: 0.5417
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 0.8439 - accuracy: 0.6833
Epoch 9/200
4/4 [==============================] - 0s 3ms/step - loss: 0.8027 - accuracy: 0.6667
Epoch 10/200
4/4 [==============================] - 0s 3ms/step - loss: 0.7667 - accuracy: 0.6667
Epoch 11/200
4/4 [===========

4/4 [==============================] - 0s 4ms/step - loss: 0.1106 - accuracy: 0.9833
Epoch 85/200
4/4 [==============================] - 0s 5ms/step - loss: 0.1098 - accuracy: 0.9750
Epoch 86/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1065 - accuracy: 0.9750
Epoch 87/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1068 - accuracy: 0.9833
Epoch 88/200
4/4 [==============================] - 0s 5ms/step - loss: 0.1018 - accuracy: 0.9833
Epoch 89/200
4/4 [==============================] - 0s 4ms/step - loss: 0.1029 - accuracy: 0.9750
Epoch 90/200
4/4 [==============================] - 0s 4ms/step - loss: 0.1008 - accuracy: 0.9833
Epoch 91/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0975 - accuracy: 0.9833
Epoch 92/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0941 - accuracy: 0.9833
Epoch 93/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0934 - accuracy: 0.9833
Epoch 94/200
4/4 [===============

4/4 [==============================] - 0s 2ms/step - loss: 0.0419 - accuracy: 0.9917
Epoch 168/200
4/4 [==============================] - 0s 6ms/step - loss: 0.0427 - accuracy: 0.9917
Epoch 169/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0411 - accuracy: 0.9917
Epoch 170/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0421 - accuracy: 0.9917
Epoch 171/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0412 - accuracy: 0.9917
Epoch 172/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0415 - accuracy: 0.9917
Epoch 173/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0409 - accuracy: 0.9917
Epoch 174/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0402 - accuracy: 0.9917
Epoch 175/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0415 - accuracy: 0.9917
Epoch 176/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0412 - accuracy: 0.9917
Epoch 177/200
4/4 [=====

In [33]:
# 학습평가
loss, accuracy = model.evaluate(test_X, test_y, verbose=0)
print(f'accuracy : {accuracy}')

accuracy : 0.9333333373069763
